In [1]:
import sys
import os
import cv2
import numpy as np
import torch
import yaml
import time
import glob
sys.path.append("../0.mc_utils/")
from pipeline.global_cfg import CFG_SINGLE
from pipeline.single_x2ds_to_A import SingleOP25bToA
from alimqtt.mqtt_server import Server

sys.path.append("../6.workspace/")
from detector2d import detector2d

>>>syp_hmr_e2e path:D:\XiaHaiPeng\20200520-3dat-climbing-v2.1\0.mc_utils\syp_hmr_e2e
classes: [0]
confidence_threshold: 0.6
device: cuda:0
framesize: 320
iou_threshold: 0.6
model: ../12.models/yolov5/yolov5s.pt
>>>YoloV4 loaded D:\XiaHaiPeng\20200520-3dat-climbing-v2.1\12.models\yolov5\yolov5s.pt making a random inference.
basic:
  crop_scale: (1.5, 1.1)
  down_scale: 4.0
  hmap_size: (56, 56)
  out_size: (224, 224)
  xyxy_conf_thresh: 0.1
conf_reader:
  conf_threashold: 0.1
  sigma: 7.0
device: cuda:0
model: ../12.models/hmr_e2e/20200904-cpn50-224x224j20-climb-0.5507786451159297.pth
network:
  block_channels: 256
  hmap_channels: 20
>>>>Loaded model:../12.models/hmr_e2e/20200904-cpn50-224x224j20-climb-0.5507786451159297.pth making a random inference.


In [2]:
myserver_one = Server(Server_id='Ali_server_0',HOST ='127.0.0.1',port=12222,keeplive=3000,camera_number=0)
myserver_two = Server(Server_id='Ali_server_1',HOST ='127.0.0.1',port=12222,keeplive=3000,camera_number=1)

Connected with result code: 0
Connected with result code: 0


In [3]:
CFG_SINGLE.defrost()
##OP setting
CFG_SINGLE.OP.net_resolution=304
CFG_SINGLE.OP.num_people_max=1
CFG_SINGLE.OP.threshold = 0.1
CFG_SINGLE.OP.fps_max = 30

##2D->3D posing
CFG_SINGLE.HMR.fix_joints=[22,23]

##REID
CFG_SINGLE.REID.max_hp = 300
# CFG_SINGLE.REID.max_cost = 1e4

##SMOOTHING and common connfig
CFG_SINGLE.enable_pose_smoothing = False
CFG_SINGLE.traj_offset = [0.0,-1.2,0.0]
CFG_SINGLE.frame_scale = 0.8
CFG_SINGLE.K = [1145,1145,1920/2,1080/2]

CFG_SINGLE.freeze()
#print(CFG_SINGLE)
offline_processor  = SingleOP25bToA(2,1.2,0,0.7,0.8,CFG_SINGLE)
mqtt_video_name,mqtt_data,mqtt_index,over_video_name=[],[],[],[]
import glob
import shutil
def remove_name(videoname):
    if videoname!='':
        uid = os.path.basename(videoname)[18:-4]
        path = 'D:/AliClimbingData/'
        copy_dir = 'D:/del_userdata/'
        for name in glob.glob(path+"*"):
            name_ = name.replace('\\','/')
            if os.path.isdir(name_):
                if name_.find(uid)!=-1:
                    #src_dir = name_.replace(os.path.basename(name_),'/')
                    move_dir = name_.replace(path,copy_dir)
                    shutil.move(name_,move_dir)
                    #shutil.rmtree(name_)
                    #os.removedirs(name)
                    print('uid重复，move文件夹：%s'%name_)
mqtt_video_name,mqtt_data,mqtt_index,over_video_name=[],[],[],[]

>>>Loaded HMR from:../12.models/mchmr2/20200516-hmrzero-b4f2048s10-vposer0420.pth
>>>Loaded VPoser from:../12.models/mchmr2/20200420-vposer1024-lt32b3.pth


In [ ]:
climbingData='D:/AliClimbingData'
import time
while True:
    #print(mqtt_video_name)
    score=[]
    #通信获得需要处理的数据
    if not myserver_one.mqtt_queue.empty():
        data = myserver_one.mqtt_queue.get()
        #print(data)
        for k,v in data.items():
            if k not in mqtt_video_name:
                mqtt_video_name.append(k)
                mqtt_index.append(v)
    if not myserver_two.mqtt_queue.empty():
        data = myserver_two.mqtt_queue.get()
        #print(data)
        for k,v in data.items():
            if k not in mqtt_video_name:
                mqtt_video_name.append(k)
                mqtt_index.append(v)
            #print(v)
    #开始处理
    for videoname in mqtt_video_name:
        #print(data)
        #t = time.time()
#         if not os.path.isdir(video_dir_name):
#             os.mkdir(video_dir_name)
        if videoname not in over_video_name :
            remove_name(videoname)
            video_dir_name = videoname.replace(os.path.basename(videoname),'')
            if not os.path.exists(videoname):
                print('no find %s'%videoname)
                continue
            test_time =time.time()
            path = os.path.abspath(os.path.dirname(videoname)+os.path.sep+".")
            json_path = video_dir_name+'op25b.npz'
            #print(json_path)
            #if os.path.isfile(json_path):
            #    pass
            #else:
            op25b={}
            ret = detector2d(videoname)
            #print(ret)
            if ret=={}:
                print('nofind people!')
                over_video_name.append(videoname)
                continue 
            for k,v in ret.items():
                op25b[k]=v
            np.savez(json_path,op25b=op25b)
            print('2D处理时间：%.2f'%(time.time()-test_time))
            test_time =time.time()
            src_video=videoname.replace('\\','/')
            backup_video = (video_dir_name+'back.mp4').replace('\\','/')
            
           
            output_video_dir = climbingData+'/'+os.path.basename(videoname).replace('.mp4','/')
            output_video=(output_video_dir+os.path.basename(videoname))
            if not os.path.isdir(output_video_dir):
                os.mkdir(output_video_dir)
            #score= [src_video,[200,300,400],backup_video,output_video,'D:/XiaHaiPeng/output/test/op25b.npz']
            #video_path,key_index,std_pose_index,std_pose_path,kp2ds_path
            #std_pose_index,std_pose_path = 
            kp2ds_path = backup_video.replace(os.path.basename(backup_video),'op25b.npz')
            #print()
            score=[src_video,mqtt_index[mqtt_video_name.index(videoname)],[90,166,285],'../../0000/op25b.npz',kp2ds_path]
            #print('输出文件夹：%s'%output_video_dir)
            #print(score)
            #print(src_video,backup_video,output_video)
            offline_processor(score,
                            src_video,
                            backup_video,
                            output_video,
                            x2ds_npz_fname=json_path,
                            interval=1,
                            resize_scale=0.8)
            print('后期处理时间：%.2f'%(time.time()-test_time))
            if videoname not in over_video_name:
                over_video_name.append(videoname)
            massage=('待处理:%d己处理:%d'%((len(mqtt_video_name)-len(over_video_name)),len(over_video_name)))
            print(massage)
            #print(time.time() - t)
    

uid-k4IvmBnIlgKthhab 0 Ali_client_1
D:/XiaHaiPeng/output/2020-09-26_153536_uid-k4IvmBnIlgKthhab/
D:/XiaHaiPeng/output/2020-09-26_153536_uid-k4IvmBnIlgKthhab/2020-09-26_153536_uid-k4IvmBnIlgKthhab.mp4 25 (1920, 1080)
uid-k4IvmBnIlgKthhab 1 Ali_client_1
2D处理时间：94.17
uid-qtgUlZUAabLgnvbG 0 Ali_client_0
D:/XiaHaiPeng/output/2020-09-26_153623_uid-qtgUlZUAabLgnvbG/
D:/XiaHaiPeng/output/2020-09-26_153623_uid-qtgUlZUAabLgnvbG/2020-09-26_153623_uid-qtgUlZUAabLgnvbG.mp4 25 (1920, 1080)
uid-qtgUlZUAabLgnvbG 1 Ali_client_0
uid-k4IvmBnIlgKthhab 9 Ali_client_1
[102, 665, 835, 1089, 1225, 1281]
(1587, 25, 3)
(1587, 25, 3)
(254, 25, 3)
(254, 25, 3)
uid-qtgUlZUAabLgnvbG 2 Ali_client_0
[0.8389830555318396, 0.8389830555318396, 0.8389830555318396]
3d计算时间：3.75 画图时间：4.34 打分时间：1.59
uid-qtgUlZUAabLgnvbG 3 Ali_client_0

>>>Cleared ReID dict instances.
后期处理时间：35.16
待处理:-10己处理:215
uid-qtgUlZUAabLgnvbG 4 Ali_client_0
uid-qtgUlZUAabLgnvbG 5 Ali_client_0
uid-qtgUlZUAabLgnvbG 6 Ali_client_0
uid-qtgUlZUAabLgnvbG 7 Al

C:\Users\mc\AppData\Roaming\Python\Python37\site-packages\scipy-1.1.0-py3.7-win-amd64.egg\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
C:\Users\mc\AppData\Roaming\Python\Python37\site-packages\scipy-1.1.0-py3.7-win-amd64.egg\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


[0.6617789317201501, 0.6617789317201501, 0.6617789317201501]
3d计算时间：3.61 画图时间：4.19 打分时间：3.18

>>>Cleared ReID dict instances.
后期处理时间：34.35
待处理:-13己处理:218
uid-7bI8tJJatJHdFQEy 0 Ali_client_1
D:/XiaHaiPeng/output/2020-09-26_154229_uid-7bI8tJJatJHdFQEy/
D:/XiaHaiPeng/output/2020-09-26_154229_uid-7bI8tJJatJHdFQEy/2020-09-26_154229_uid-7bI8tJJatJHdFQEy.mp4 25 (1920, 1080)
uid-7bI8tJJatJHdFQEy 1 Ali_client_1
uid-DtLuMDkR2mNXOtZJ 0 Ali_client_0
D:/XiaHaiPeng/output/2020-09-26_154243_uid-DtLuMDkR2mNXOtZJ/
D:/XiaHaiPeng/output/2020-09-26_154243_uid-DtLuMDkR2mNXOtZJ/2020-09-26_154243_uid-DtLuMDkR2mNXOtZJ.mp4 25 (1920, 1080)
uid-DtLuMDkR2mNXOtZJ 1 Ali_client_0
uid-7bI8tJJatJHdFQEy 2 Ali_client_1
uid-DtLuMDkR2mNXOtZJ 2 Ali_client_0
uid-7bI8tJJatJHdFQEy 3 Ali_client_1
uid-DtLuMDkR2mNXOtZJ 3 Ali_client_0
uid-7bI8tJJatJHdFQEy 4 Ali_client_1
uid-DtLuMDkR2mNXOtZJ 4 Ali_client_0
uid-DtLuMDkR2mNXOtZJ 5 Ali_client_0
uid-DtLuMDkR2mNXOtZJ 6 Ali_client_0
uid-7bI8tJJatJHdFQEy 5 Ali_client_1
uid-7bI8tJJatJHdFQ

In [ ]:
# #over_video_name=[]
# for name in glob.glob('D:/XiaHaiPeng/output/*'):
#     name = name.replace('\\','/')
#     #print(name)
#     for videoname in glob.glob(name+"/*.mp4"):
#         #print(videoname)
#         if videoname.find('back.mp4')==-1:
#             over_video_name.append(videoname.replace('\\','/'))
# print(over_video_name)

In [20]:
mqtt_video_name,mqtt_index=[],[]
for name in glob.glob('D:/XiaHaiPeng/offline/*'):
    name = name.replace('\\','/')
    #print(name)
    for videoname in glob.glob(name+"/*.mp4"):
        #print(videoname)
        if videoname.find('back.mp4')==-1:
            mqtt_video_name.append(videoname.replace('\\','/'))
            mqtt_index.append([])
print(mqtt_video_name)
print(len(mqtt_video_name))
#over = 'D:/XiaHaiPeng/output/2020-09-24_142714_uid-Xwdr6EGVmgEzjB5n/2020-09-24_142714_uid-Xwdr6EGVmgEzjB5n.mp4'
#over_video_name.append(over)

['D:/XiaHaiPeng/offline/2020-09-26_141001_uid-kedhnEDrAyeAp5y8/2020-09-26_141001_uid-kedhnEDrAyeAp5y8.mp4']
1


In [22]:
over = 'D:/XiaHaiPeng/output/2020-09-26_150012_uid-850lIxIFKShnGYlP/2020-09-26_150012_uid-850lIxIFKShnGYlP.mp4'
del mqtt_index[mqtt_video_name.index(over)]
mqtt_video_name.remove(over)
over_video_name.append(over)

In [13]:
list_=[]
copy_name = 'D:/XiaHaiPeng/output/'
for name in glob.glob('D:/XiaHaiPeng/offline/*'):
    name = name.replace('\\','/')
    #print(name)
    for videoname in glob.glob(name+"/*.mp4"):
        #print(videoname)
        if videoname.find('back.mp4')==-1:
            
            over_video_name.append(videoname.replace('\\','/').replace('D:/XiaHaiPeng/offline/',copy_name))
#over = 'D:/XiaHaiPeng/output/2020-09-25_084207_uid-cRt4o8fqmVUwHt9b/2020-09-25_084207_uid-cRt4o8fqmVUwHt9b.mp4'
print(over_video_name)

['D:/XiaHaiPeng/output/2020-09-25_100928_uid-sQS49Juye00C8R0A/2020-09-25_100928_uid-sQS49Juye00C8R0A.mp4', 'D:/XiaHaiPeng/output/2020-09-25_101219_uid-BEfK8bjuoyscFlVK/2020-09-25_101219_uid-BEfK8bjuoyscFlVK.mp4', 'D:/XiaHaiPeng/output/2020-09-25_101342_uid-fLoTgXgb7pm16GR1/2020-09-25_101342_uid-fLoTgXgb7pm16GR1.mp4', 'D:/XiaHaiPeng/output/2020-09-25_101532_uid-QbuoHgxeOrqZEZ5n/2020-09-25_101532_uid-QbuoHgxeOrqZEZ5n.mp4', 'D:/XiaHaiPeng/output/2020-09-25_101652_uid-EPwYtMQSyhb8xOqX/2020-09-25_101652_uid-EPwYtMQSyhb8xOqX.mp4', 'D:/XiaHaiPeng/output/2020-09-25_101812_uid-epypcjYbdl9HSMqt/2020-09-25_101812_uid-epypcjYbdl9HSMqt.mp4', 'D:/XiaHaiPeng/output/2020-09-25_101915_uid-2zsWcWBOnZalFTmh/2020-09-25_101915_uid-2zsWcWBOnZalFTmh.mp4', 'D:/XiaHaiPeng/output/2020-09-25_102154_uid-2niaiRBS22lS01NQ/2020-09-25_102154_uid-2niaiRBS22lS01NQ.mp4', 'D:/XiaHaiPeng/output/2020-09-25_102128_uid-hFrPRt2sR7QA5XG0/2020-09-25_102128_uid-hFrPRt2sR7QA5XG0.mp4', 'D:/XiaHaiPeng/output/2020-09-25_102641_uid-X

In [1]:
np.savetxt('offline_undeal.txt',mqtt_video_name)
np.savetxt('offline_deal.txt',over_video_name)
np.savetxt('offline_index.txt',mqtt_index)

NameError: name 'np' is not defined

In [ ]:
# climbingData='D:/AliClimbingData'
# mqtt_video_name,mqtt_data=[],[]
# for name in glob.glob('D:/XiaHaiPeng/output/'+'*'):
#     for name_ in glob.glob(name+'/*'):
#         if name_.find('back.mp4')==-1 and name_.find('npz')==-1 and name_.find('mp4')!=-1:
#             mqtt_video_name.append(name_.replace('\\','/'))
#         if name_.find('op25b.npz')!=-1:
#             mqtt_data.append(name_.replace('\\','/'))
# print(len(mqtt_video_name),len(mqtt_data))
# print(mqtt_video_name,mqtt_data)
# over_video_name = []
# offline_processor  = SingleOP25bToA(2,1.2,0,0.5,0.8,CFG_SINGLE)

